In [1]:
import pandas as pd
from konlpy.tag import Twitter

In [2]:
# 사진 제외 : 87%
# 사진 포함 : 88%

In [3]:
df_naver = pd.read_csv('A_V_Adj_C_df_naver_post_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep = pd.read_csv('A_V_Adj_C_df_dbdbdeep_post_table.csv',sep='\t',encoding='utf-8')
df_naver_page = pd.read_csv('A_V_Adj_C_df_naver_page_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep_page = pd.read_csv('A_V_Adj_C_df_dbdbdeep_page_table.csv',sep='\t',encoding='utf-8')

In [4]:
df_dbdbdeep.groupby(by='Category')['label'].count()

Category
IT·컴퓨터       514
건강·의학        917
공연·전시        378
교육·학문        732
국내여행          80
드라마·방송       383
등산·낚시·레저      39
만화·애니         30
맛집           662
사진            31
스포츠           64
시사·인문·경제      95
어학·외국어       179
와인·술          68
육아·결혼         55
자동차          330
차·커피·디저트     569
패션·뷰티       1238
Name: label, dtype: int64

In [5]:
df_naver.groupby(by='Category')['label'].count()

Category
IT·컴퓨터       476
건강·의학        565
공연·전시        532
교육·학문        492
국내여행         475
드라마·방송       258
등산·낚시·레저     232
만화·애니        653
맛집          1033
사진          1682
스포츠          841
시사·인문·경제     565
어학·외국어       162
와인·술          80
육아·결혼        257
자동차          643
차·커피·디저트     254
패션·뷰티        742
Name: label, dtype: int64

In [6]:
df_naver = df_naver.dropna(subset=['Embedding_text'])
df_dbdbdeep = df_dbdbdeep.dropna(subset=['Embedding_text'])

In [7]:
def Embedding_text_reduction(Embedding_text):
    if len(Embedding_text.split(' '))> 55:
        return " ".join(Embedding_text.split(' ')[:-28])
    elif len(Embedding_text.split(' '))>45:
        return " ".join(Embedding_text.split(' ')[:-12])
    elif len(Embedding_text.split(' '))>35:
        return " ".join(Embedding_text.split(' ')[:-5])
    else:
        return " ".join(Embedding_text.split(' '))

In [8]:
# 사진 포함 33
df_naver['Embedding_text'] = df_naver['Embedding_text'].apply(lambda x : Embedding_text_reduction(x) )

In [9]:
from sklearn.utils import shuffle
df_naver = shuffle(df_naver)

In [10]:
df_dbdbdeep.shape

(6208, 19)

In [11]:
# df_naver = df_naver.iloc[:6208]

In [12]:
Total = pd.concat([df_naver,df_dbdbdeep,df_naver_page,df_dbdbdeep_page])

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [13]:
Total = Total.dropna(subset=['Embedding_text'])

In [14]:
Total.shape

(38401, 19)

## Doc2vec

In [15]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
Total = Total[['Embedding_text','label']]

In [17]:
text = Total['Embedding_text'].tolist()

In [18]:
y = Total['label'].tolist()

In [19]:
sentences = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text)]

In [20]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [21]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [22]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words
    size=100,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=20,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,   # negative sampling / default 5
)

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [23]:
doc_vectorizer.build_vocab(sentences)

2018-09-05 13:19:48,871 : INFO : collecting all words and their counts
2018-09-05 13:19:48,871 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2018-09-05 13:19:48,872 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-09-05 13:19:49,100 : INFO : PROGRESS: at example #10000, processed 1469543 words (6500427/s), 971 word types, 10000 tags
2018-09-05 13:19:49,425 : INFO : PROGRESS: at example #20000, processed 3212332 words (5365839/s), 997 word types, 20000 tags
2018-09-05 13:19:49,655 : INFO : PROGRESS: at example #30000, processed 4715977 words (6578281/s), 1005 word types, 30000 tags
2018-09-05 13:19:49,855 : INFO : collected 1012 word types and 38401 unique tags from a corpus of 38401 examples and 6033405 words
2018-09-05 13:19:49,856 : INFO : Loading a fresh vocabulary
2018-09-05 13:19:49,860 : INFO : min_count=20 retains 877 unique words (86% of original 1012, drops 13

In [24]:
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d100,n10,hs,w8,mc20,s0.001,t12)


In [25]:
doc_vectorizer.corpus_count*0.03

1152.03

In [26]:
doc_vectorizer.iter

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


5

In [27]:
import time
start = time.time()
for epoch in range(10):
    doc_vectorizer.train(sentences, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time.time()
print("During Time: {}".format(end-start))

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.
2018-09-05 13:19:50,398 : INFO : training model with 12 workers on 877 vocabulary and 100 features, using sg=1 hs=1 sample=0.001 negative=10 window=8
2018-09-05 13:19:51,527 : INFO : EPOCH 1 - PROGRESS: at 8.48% examples, 176741 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:19:52,529 : INFO : EPOCH 1 - PROGRESS: at 17.24% examples, 190403 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:19:53,612 : INFO : EPOCH 1 - PROGRESS: at 27.65% examples, 197428 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:19:54,631 : INFO : EPOCH 1 - PROGRESS: at 36.76% examples, 202506 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:19:55,632 : INFO : EPOCH 1 - PROGRESS: at 43.99% examples, 202301 words/s, in_qsize 24, out_qsize 1
2018-09-05 13:19:56,641 : INFO : EPOCH 1 - PROGRESS: a

2018-09-05 13:20:30,475 : INFO : EPOCH 4 - PROGRESS: at 26.07% examples, 195175 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:20:31,526 : INFO : EPOCH 4 - PROGRESS: at 35.22% examples, 198128 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:20:32,527 : INFO : EPOCH 4 - PROGRESS: at 42.77% examples, 198160 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:20:33,539 : INFO : EPOCH 4 - PROGRESS: at 48.66% examples, 197683 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:20:34,551 : INFO : EPOCH 4 - PROGRESS: at 55.55% examples, 197460 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:20:35,558 : INFO : EPOCH 4 - PROGRESS: at 62.71% examples, 197312 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:20:36,569 : INFO : EPOCH 4 - PROGRESS: at 72.74% examples, 198344 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:20:37,586 : INFO : EPOCH 4 - PROGRESS: at 82.19% examples, 198365 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:20:38,600 : INFO : EPOCH 4 - PROGRESS: at 91.11% examples, 198019 words/s,

2018-09-05 13:21:10,309 : INFO : EPOCH 2 - PROGRESS: at 22.96% examples, 172764 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:21:11,311 : INFO : EPOCH 2 - PROGRESS: at 31.54% examples, 175822 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:21:12,319 : INFO : EPOCH 2 - PROGRESS: at 38.29% examples, 174139 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:21:13,325 : INFO : EPOCH 2 - PROGRESS: at 44.08% examples, 173193 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:21:14,343 : INFO : EPOCH 2 - PROGRESS: at 49.39% examples, 173452 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:21:15,379 : INFO : EPOCH 2 - PROGRESS: at 55.55% examples, 173215 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:21:16,381 : INFO : EPOCH 2 - PROGRESS: at 61.51% examples, 173204 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:21:17,421 : INFO : EPOCH 2 - PROGRESS: at 69.94% examples, 172120 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:21:18,447 : INFO : EPOCH 2 - PROGRESS: at 78.51% examples, 172896 words/s,

2018-09-05 13:21:53,074 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-05 13:21:53,074 : INFO : EPOCH - 4 : training on 6033405 raw words (2468422 effective words) took 15.9s, 155098 effective words/s
2018-09-05 13:21:54,106 : INFO : EPOCH 5 - PROGRESS: at 5.64% examples, 130714 words/s, in_qsize 24, out_qsize 1
2018-09-05 13:21:55,130 : INFO : EPOCH 5 - PROGRESS: at 13.12% examples, 150047 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:21:56,148 : INFO : EPOCH 5 - PROGRESS: at 20.16% examples, 154325 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:21:57,168 : INFO : EPOCH 5 - PROGRESS: at 27.42% examples, 154047 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:21:58,171 : INFO : EPOCH 5 - PROGRESS: at 34.37% examples, 156894 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:21:59,177 : INFO : EPOCH 5 - PROGRESS: at 41.11% examples, 158876 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:22:00,207 : INFO : EPOCH 5 - PROGRESS: at 45.94% examples, 158561 words/s,

2018-09-05 13:22:36,113 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-05 13:22:36,120 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-05 13:22:36,124 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-05 13:22:36,139 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-05 13:22:36,148 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-05 13:22:36,149 : INFO : EPOCH - 2 : training on 6033405 raw words (2467997 effective words) took 13.3s, 185824 effective words/s
2018-09-05 13:22:37,167 : INFO : EPOCH 3 - PROGRESS: at 6.55% examples, 152379 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:22:38,185 : INFO : EPOCH 3 - PROGRESS: at 14.90% examples, 171572 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:22:39,219 : INFO : EPOCH 3 - PROGRESS: at 23.27% examples, 177014 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:22:40,240 : INFO : EPOCH 3 - PROGRESS: at 31.64% exa

...

2018-09-05 13:28:51,218 : INFO : EPOCH 4 - PROGRESS: at 15.71% examples, 179017 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:28:52,242 : INFO : EPOCH 4 - PROGRESS: at 24.39% examples, 183664 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:28:53,254 : INFO : EPOCH 4 - PROGRESS: at 32.44% examples, 184241 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:28:54,316 : INFO : EPOCH 4 - PROGRESS: at 40.58% examples, 184837 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:28:55,339 : INFO : EPOCH 4 - PROGRESS: at 46.45% examples, 185626 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:28:56,352 : INFO : EPOCH 4 - PROGRESS: at 52.64% examples, 184837 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:28:57,367 : INFO : EPOCH 4 - PROGRESS: at 59.22% examples, 185562 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:28:58,380 : INFO : EPOCH 4 - PROGRESS: at 67.37% examples, 185193 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:28:59,407 : INFO : EPOCH 4 - PROGRESS: at 76.91% examples, 185858 words/s,

2018-09-05 13:29:30,917 : INFO : EPOCH 2 - PROGRESS: at 6.38% examples, 143969 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:29:31,952 : INFO : EPOCH 2 - PROGRESS: at 14.37% examples, 161697 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:29:32,960 : INFO : EPOCH 2 - PROGRESS: at 21.84% examples, 165162 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:29:33,983 : INFO : EPOCH 2 - PROGRESS: at 29.89% examples, 164746 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:29:35,000 : INFO : EPOCH 2 - PROGRESS: at 36.57% examples, 166208 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:29:36,006 : INFO : EPOCH 2 - PROGRESS: at 42.47% examples, 163948 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:29:37,026 : INFO : EPOCH 2 - PROGRESS: at 47.05% examples, 163701 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:29:38,054 : INFO : EPOCH 2 - PROGRESS: at 53.04% examples, 164424 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:29:39,058 : INFO : EPOCH 2 - PROGRESS: at 58.39% examples, 163582 words/s, 

2018-09-05 13:30:13,827 : INFO : EPOCH 5 - PROGRESS: at 8.14% examples, 187124 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:30:14,850 : INFO : EPOCH 5 - PROGRESS: at 16.58% examples, 190204 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:30:15,857 : INFO : EPOCH 5 - PROGRESS: at 25.22% examples, 190914 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:30:16,859 : INFO : EPOCH 5 - PROGRESS: at 34.03% examples, 195196 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:30:17,870 : INFO : EPOCH 5 - PROGRESS: at 42.18% examples, 196998 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:30:18,884 : INFO : EPOCH 5 - PROGRESS: at 47.56% examples, 195330 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:30:19,907 : INFO : EPOCH 5 - PROGRESS: at 54.28% examples, 195055 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:30:20,916 : INFO : EPOCH 5 - PROGRESS: at 61.50% examples, 195674 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:30:21,925 : INFO : EPOCH 5 - PROGRESS: at 71.65% examples, 196944 words/s, 

2018-09-05 13:30:53,822 : INFO : EPOCH 3 - PROGRESS: at 35.06% examples, 193291 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:30:54,847 : INFO : EPOCH 3 - PROGRESS: at 42.77% examples, 194198 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:30:55,858 : INFO : EPOCH 3 - PROGRESS: at 48.45% examples, 193823 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:30:56,863 : INFO : EPOCH 3 - PROGRESS: at 55.67% examples, 195385 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:30:57,889 : INFO : EPOCH 3 - PROGRESS: at 63.50% examples, 196483 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:30:58,920 : INFO : EPOCH 3 - PROGRESS: at 73.16% examples, 196261 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:30:59,922 : INFO : EPOCH 3 - PROGRESS: at 82.48% examples, 196377 words/s, in_qsize 24, out_qsize 0
2018-09-05 13:31:00,926 : INFO : EPOCH 3 - PROGRESS: at 91.88% examples, 197474 words/s, in_qsize 23, out_qsize 0
2018-09-05 13:31:01,892 : INFO : worker thread finished; awaiting finish of 11 more thre

During Time: 696.7924404144287


## Model_save

In [28]:
model_name = 'Text_Doc2vec_features_100.model'
doc_vectorizer.save(model_name)

2018-09-05 13:31:27,193 : INFO : saving Doc2Vec object under Text_Doc2vec_features_100.model, separately None
2018-09-05 13:31:27,429 : INFO : saved Text_Doc2vec_features_100.model


## model_load

In [29]:
doc_vectorizer = Doc2Vec.load(model_name)

2018-09-05 13:31:27,436 : INFO : loading Doc2Vec object from Text_Doc2vec_features_100.model
2018-09-05 13:31:27,678 : INFO : loading vocabulary recursively from Text_Doc2vec_features_100.model.vocabulary.* with mmap=None
2018-09-05 13:31:27,679 : INFO : loading trainables recursively from Text_Doc2vec_features_100.model.trainables.* with mmap=None
2018-09-05 13:31:27,680 : INFO : loading wv recursively from Text_Doc2vec_features_100.model.wv.* with mmap=None
2018-09-05 13:31:27,680 : INFO : loading docvecs recursively from Text_Doc2vec_features_100.model.docvecs.* with mmap=None
2018-09-05 13:31:27,681 : INFO : loaded Text_Doc2vec_features_100.model


In [30]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in sentences]

In [31]:
Text_df = pd.DataFrame({'word':X_train,'label':y})

In [32]:
from sklearn.utils import shuffle
Text_df = shuffle(Text_df)

In [33]:
Text_df = Text_df.dropna()

In [34]:
X_train = Text_df['word'].tolist()
y = Text_df['label'].tolist()

## Data_split

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train , y , test_size=0.33, random_state=42)

## SVM

In [36]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.8743718592964824


# MLP

In [37]:
from sklearn.neural_network import MLPClassifier

In [38]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(50,),
    max_iter=10,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [39]:
start = time.time()
mlp_clf.fit(x_train, y_train)
end = time.time()
print('Time: {:f}s'.format(end-start))

Iteration 1, loss = 0.41765932
Iteration 2, loss = 0.29748632
Iteration 3, loss = 0.28318378
Iteration 4, loss = 0.26883099
Iteration 5, loss = 0.25527985
Iteration 6, loss = 0.24242841
Iteration 7, loss = 0.23190347
Iteration 8, loss = 0.22318658
Iteration 9, loss = 0.21476358
Iteration 10, loss = 0.20640816
Time: 0.316915s


C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [40]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))

테스트 정확도: 0.901
